In [1]:
import pandas as pd
import sys
import io
import time
from gurobipy import *
from clean_data_cor import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def read_data(path_dem:str, path_preco:str, path_rota:str):

    demanda = pd.read_csv(path_dem)
    preco = pd.read_csv(path_preco)
    rota0 = pd.read_csv(path_rota)
    rota1 = eval(rota0['Route'][0])

    return demanda, preco, rota1


def create_sets(demanda, preco_, rota1, perio=0):

    if perio != 0:
        periodo_lim = sorted(demanda['DBD'].unique().tolist())[:perio]
        demanda = demanda[demanda['DBD'].isin(periodo_lim)]

    rota = [0] + rota1
    
    I, J, OD, V,  T, stations, VK, P, d, dd, indexCombiDem, indexCombiDem0 = clean_data2(demanda, preco_)

    AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
    NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

    I = [i for i in rota if i in I]
    I2 = [0] + I# tambem se toma para el para tudo da restriccion de fluxo
    J = [i for i in rota if i in J]
    n = len(rota)-1

    # listas de trechos contenidos dentro de otros trechos
    BR = {}
    for i,j in NAD:
        listTemp = list(combinations(rota1[rota1.index(i):rota1.index(j)+1], 2))
        listTemp =  {(ii,jj) for ii,jj in listTemp if (ii,jj) in OD and (ii, jj) != (i,j)} #rota1.index(jj) == rota1.index(ii)+1 and 
        BR[(i,j)] = listTemp

    #indices
    index = [(i,j,v,k,t) for i,j,v,k,t in  indexCombiDem if (i,j) in BR.keys()]
    
    return I, I2, J, OD, NAD, V,  T, stations, VK, P, d, dd, n, BR, index, indexCombiDem, indexCombiDem0, demanda


def create_model(I, rota, VK, NAD, BR, P, Q, d, index, indexCombiDem, indexCombiDem0, demanda, dd):

    model = Model("Modelo 1.1.1")

    # variables de decicion
    X = model.addVars(indexCombiDem0, vtype=GRB.INTEGER , name="X")
    Y = model.addVars(indexCombiDem, vtype=GRB.INTEGER , name="Y")
    A = model.addVars(rota, vtype=GRB.INTEGER , name="A")
    BNA = model.addVars(index, vtype=GRB.BINARY , name="BNA")


    # funcion objetivo
    model.setObjective(
        quicksum(P[(i,j,v,k)]*X[(i,j,v,k,t)] for i,j,v,k,t in indexCombiDem),
        sense = GRB.MAXIMIZE
    )

    # restricciones
    for i in I:

        # restricao .2
        model.addConstr(
            A[i] == A[rota[rota.index(i)-1]] - 
            quicksum(X[rota[rota.index(i)-1],j,v,k,t] for i_,j,v,k,t in indexCombiDem0 if (rota[rota.index(i)-1] == i_)) + 
            quicksum(X[i_,j,v,k,t] for i_,j,v,k,t in indexCombiDem0 if (j == i)),
            name=f"Dispo_{i}"
        )

        # restricao .3
        model.addConstr(
            quicksum((X[i,j,v,k,t]) for i_,j,v,k,t in indexCombiDem if i == i_) <= A[i],
            name=f"Cap_{i}"
        )

        # restricao .6
        model.addConstr(
            quicksum(Y[i,j,v,VK[v][0],t] for i_,j,v,k,t in indexCombiDem if (i == i_) and (VK[v][0] == k)) <= Q, 
            name=f"AuthoCap_{i}"
        )


    for i,j,v,k,t in indexCombiDem:

        VK_ = demanda.loc[(demanda["Origin"]==i) & (demanda["Destination"]==j) & (demanda["Vagon"]==v) & (demanda["DBD"]==t)]["Class"].to_list()
    
        pos_k = VK_.index(k)
        last_k = VK_[-1]

        # restricao .5
        model.addConstr(
            X[i,j,v,k,t] <= d[i,j,v,k,t],
            name = f"Assig_({i},{j},{v},{k},{t})"
        )

        # # restricao .demanda comportamental
        # if pos_k >= 1:
        #     model.addConstr(
        #         quicksum(X[i,j,v,kk,t] for kk in VK_[0:pos_k+1]) <= d[i,j,v,k,t],
        #         name = f"DemComp({i},{j},{v},{k},{t})"
        #     )


        if k != last_k:

            # # restricao .5 [1ra parte] modifcacion de la demanda con el porcentaje
            # model.addConstr(
            #     X[i,j,v,k,t] <= d[i,j,v,k,t]*(dd[i,j,v,k,t] / d[i,j,v,last_k,t]),
            #     name = f"Assig1_({i},{j},{v},{k},{t})"
            # )

            # restricao .4
            model.addConstr(
                Y[i,j,v,k,t] >= Y[i,j,v,VK_[pos_k+1],t], 
                name=f"Classe_({i},{j},{v},{k},{t})"
            )

            # # restricao .4.2 [jerarquia para los assigments] =========================================
            # model.addConstr(
            #     X[i,j,v,k,t] <= X[i,j,v,VK_[pos_k+1],t], 
            #     name=f"JerarAssig_({i},{j},{v},{k},{t})"
            # )

            # restricao .8
            model.addConstr(
                Y[i,j,v,k,t] >=  X[i,j,v,k,t] + Y[i,j,v,VK_[pos_k+1],t],
                name=f"Autho_({i},{j},{v},{k},{t})"
            )
        else:

            # restricao .7
            model.addConstr(
                Y[i,j,v,k,t] >=  X[i,j,v,k,t],
                name=f"Autho_({i},{j},{v},{k},{t})"
            )

            # # restricao .5 [2da parte] modifcacion de la demanda con el porcentaje
            # model.addConstr(
            #     X[i,j,v,k,t] <= dd[i,j,v,k,t] + (d[i,j,v,last_k,t] - quicksum( d[i,j,v,kk,t]*(dd[i,j,v,kk,t] / d[i,j,v,last_k,t]) for kk in VK_)),
            #     name = f"Assig2_({i},{j},{v},{k},{t})"
            # )


        if (i,j) in NAD:
            # aqui el (i,j)=(o,d)

            # restricao .9.1
            model.addConstr(
                BNA[i,j,v,k,t] <= Y[i,j,v,k,t],
                # name = f"activ_bin_autho_low_({o},{d_},{v},{k},{t})"
            )
            
            # restricao .9.2
            model.addConstr(
                Y[i,j,v,k,t] <= Q*BNA[i,j,v,k,t],
                # name = f"activ_bin_autho_top_({o},{d_},{v},{k},{t})"
            )

            for ii,jj,vv,kk,tt in indexCombiDem:
                if (ii,jj) in BR[(i,j)] and v == vv and k == kk and t == tt:
                    # restricao .10.1
                    model.addConstr(
                        Y[ii,jj,vv,kk,tt] <= Q*BNA[i,j,vv,kk,tt],
                        # name = f"pru1({o},{d_},{i},{j},{v},{k},{t})"
                    )

                    # restricao .10.2
                    model.addConstr(
                        BNA[i,j,vv,kk,tt] <= Y[ii,jj,vv,kk,tt],
                        # name = f"pru2({o},{d_},{i},{j},{v},{k},{t})"
                    )


    for i,j,v,k,t in indexCombiDem0:
        # restricao .11
        if i == 0:
            model.addConstr(
                X[i,j,v,k,t] == 0,
                name = f"Assig_({0},{j},{v},{k},{t})"
            )

    # restricao .12
    model.addConstr(
        A[0] == Q,
        name = f"Cap_0")

    # Optimizar o modelo
    model.optimize()

    return model, A, X, Y, BNA


def save_solution(model, BR, P, d, X, Y, BNA, perio, instance, indexCombiDem):
    print('Valor da função objetivo: ', str(model.ObjVal) )
    print('')
    lista = []
    for i,j, v, k, t in indexCombiDem:
        if (i,j) in BR.keys():
            lista.append([i+'-'+j,i,j,v, k, t, P[i,j,v,k], d[i,j,v,k,t], X[i,j,v,k,t].X, Y[i,j,v,k,t].X , BNA[i,j,v,k,t].X ])
        else:
            lista.append([i+'-'+j,i,j,v, k, t, P[i,j,v,k], d[i,j,v,k,t], X[i,j,v,k,t].X, Y[i,j,v,k,t].X , -1 ])

    a = pd.DataFrame(lista, columns=['o-d',"Origen","Destino",'Vagon','classe','Periodo','Preco','Demanda','Assignments','Authorizations','Binaria'])
    
    a.to_excel('C:/Users/LAB_C/Documents/wilmer/tesis/ResultadosNew/RestrSemDemanComp/modelo_'+instance+'_t'+str(perio)+'.xlsx', index=False)

    return a


In [3]:
# parametros
instances = ['instancia1', 'instancia2','instancia3']#
Qs = [561, 565, 565]#
num_periodos = [0]#

In [4]:
start_time_exp = time.time()
for instance in instances:
    
    path_dem = 'C:/Users/LAB_C/Documents/wilmer/tesis/Instancias/'+instance+'/demanda.csv'
    path_preco = 'C:/Users/LAB_C/Documents/wilmer/tesis/Instancias/'+instance+'/preco.csv'
    path_rota = 'C:/Users/LAB_C/Documents/wilmer/tesis/Instancias/'+instance+'/rota.csv'
    
    Q = Qs[instances.index(instance)]

    # chamar dados
    demanda, preco_, rota1 = read_data(path_dem, path_preco, path_rota)

    for num_periodo in num_periodos:
        # Criar Conjuntos
        I, I2, J, OD, NAD, V,  T, stations, VK, P, d, dd, n, BR, index, indexCombiDem, indexCombiDem0, demandaFil= create_sets(demanda, preco_, rota1, num_periodo)

        # criar modelo
        start_time = time.time()
        model, A, X, Y, BNA = create_model(I, I2, VK, NAD, BR, P, Q, d, index, indexCombiDem, indexCombiDem0, demandaFil, dd)
        end_time = time.time()
        tempo = end_time - start_time

        # Redirigir la salida estándar a un archivo
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        # Optimizar o modelo
        model.optimize()

        # Salvar solucao
        save_solution(model, BR, P, d, X, Y, BNA, num_periodo, instance, indexCombiDem)
        print('Tempo para criar o modelo: ', tempo)

        # Volver a la salida estándar
        sys.stdout = old_stdout

        # Obtener la salida de la optimización
        output = new_stdout.getvalue()

        # Guardar la salida en un archivo de texto
        with open('C:/Users/LAB_C/Documents/wilmer/tesis/ResultadosNew/RestrSemDemanComp/modelo1_'+instance+'_t'+str(num_periodo)+'.txt', 'w') as f:
            f.write(output)

end_time_exp = time.time()
all_time = end_time_exp - start_time_exp


Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-31
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 61295 rows, 57329 columns and 171719 nonzeros
Model fingerprint: 0x4f13a407
Variable types: 0 continuous, 57329 integer (1088 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [6e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Presolve removed 59697 rows and 55680 columns
Presolve time: 0.02s
Presolved: 1598 rows, 1649 columns, 5987 nonzeros
Variable types: 0 continuous, 1649 integer (1196 binary)
Found heuristic solution: objective 170484.85000

Explored 1 nodes (0 simplex iterations) in 0.06 seconds (0.07 work units)
Thread count was 24 (of 24 available processors)

So

In [5]:
# demanda.loc[(demanda.Origin=="S8") & (demanda.Destination=="S25") &  (demanda.Vagon=="Z")]
# preco.loc[(demanda.Origin=="S8") & (demanda.Destination=="S25") &  (demanda.Vagon=="Z")]
# d[('S8', 'S25', 'Z', 8, 94)]

# ('S8', 'S25', 'Z', 8, 94) in 


In [6]:
# # chamar dados
# demanda, preco_, rota1 = read_data(path_dem, path_preco, path_rota)

# # Criar Conjuntos
# I, I2, J, OD, NAD, V,  T, stations, VK, P, d, n, BR, rota, index0, index = crate_sets(demanda, preco_, rota1, num_periodo)

# # criar modelo
# model, A, X, Y, BNA = create_model(I, I2, J, V, VK, T, OD, NAD, BR, P, Q, d, index0, index, rota)

# # Redirigir la salida estándar a un archivo
# old_stdout = sys.stdout
# new_stdout = io.StringIO()
# sys.stdout = new_stdout

# # Optimizar o modelo
# model.optimize()

# # Salcar solucao
# save_solution(model, OD, V, VK, T, BR, P, d, X, Y, BNA, num_periodo)

# # Volver a la salida estándar
# sys.stdout = old_stdout

# # Obtener la salida de la optimización
# output = new_stdout.getvalue()

# # Guardar la salida en un archivo de texto
# with open('modelo1_',instance+'_t'+str(num_periodo)+'.txt', 'w') as f:
#     f.write(output)

In [7]:
# import random 
# rota1 = ['uno','dos','tres', 'cuatro', 'cinco']
# rota = [0] + rota1

# OD = [('uno','dos'),
#            ('uno','tres'),
#            ('uno','cuatro'),
#            ('uno','cinco'),
#            ('dos','tres'),
#            ('dos','cuatro'),
#            ('dos','cinco'),
#            ('tres','cuatro'),
#            ('tres','cinco'),
#            ('cuatro','cinco'),]

# AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
# NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

# I = ['uno','dos','tres', 'cuatro']
# I2 = [0] + I

# J = ['dos','tres','cuatro', 'cinco']
# V = ['p','z']
# T = [0,1]
# Q = 700 
# n = len(rota) - 1 
# VK = {'p':[1,5,7],'z':[1,3,7]}

# d = {(i,j,v,k,t): np.random.randint(1,100) for i,j in OD for v in V for k in VK[v] for t in T }

# P = {}
# for i,j in OD:
#     for v in V:
#         aleat = sorted(random.sample(range(50, 2000 + 1), len(VK[v])), reverse=True)
#         for k,valor in enumerate(aleat):
#             P[(i,j,v,VK[v][k])] = valor

# # corrigir instancias
# d = {(i,j,v,k,t): d[(i,j,v,k,t)] if (i,j,v,k,t) in d else 0  for i,j in OD for v in V for k in VK[v] for t in T}
# P = {(i,j,v,k): P[(i,j,v,k)] if (i,j,v,k) in P else 0  for i,j in OD for v in V for k in VK[v] }